https://assaeunji.github.io/machine%20learning/2020-11-29-implicitfeedback/

# test 3주 정합성 검증
- 

# 추천 모델
- ALS MF, LMF, MP (총 3개)
- 총 3개의 추천을 진행하며 MF와 LMF 의 경우 콜드스타트 유저(신규 유저)인 경우 MP로 추천 진행

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import scipy.sparse as sp
from implicit.lmf import LogisticMatrixFactorization as LMF

/home/user_3/medistream-recsys/Script


# 1.Dataload

In [2]:
# products name 확인 용
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 627814.50it/s]


In [3]:
df = pd.read_json('/fastcampus-data/select_column_version_4.json')

In [4]:
df['date_paid'] = pd.to_datetime(df['date_paid'])
all_df = df.copy()

In [5]:
def product_name_fill(product_name_preprocess_df):
    # 각 마지막 product_ids, name으로 채우기
    product_ids_to_name = {}
    for idx, row in product_name_preprocess_df.iterrows():
        product_ids_to_name[row.product_ids] = row.name_x
    product_name_preprocess_df['name_x'] = product_name_preprocess_df['product_ids'].apply(lambda x: product_ids_to_name[x])

    name_to_product_ids = {}
    for idx, row in product_name_preprocess_df.iterrows():
        name_to_product_ids[row.name_x] = row.product_ids
    product_name_preprocess_df['product_ids'] = product_name_preprocess_df['name_x'].apply(lambda x: name_to_product_ids[x])
    return product_name_preprocess_df

In [6]:
def promotion_proprof(df):
    from datetime import datetime

    preprocessed_book_df_date = df.copy()

    promotion_book_df = preprocessed_book_df_date[preprocessed_book_df_date['date_paid'] >= '2022-01-01']
    promotion_book_df['date_paid_date'] = promotion_book_df['date_paid'].dt.date
    promotion_book_df['date_paid_week'] = promotion_book_df['date_paid_date'].apply(lambda x: x.isocalendar()[1])

    promotion_dict = {
        2:['트리거포인트 침치료'],
        3:['藥徵, 약의 징표','파킨슨병 한의진료','침의 과학적 접근의 이해','길익동동','Medical acupuncture 침의 과학적 접근과 임상활용',\
          '동의보감 약선','수화론(水火論)'],
        4:['실전한약가이드','음양승강으로 해석하는 사상의학: 생리병리'],
        5:['음양승강으로 해석하는 사상의학: 생리병리'],
        6:['윤상훈·권병조의 알짜 근육학','임상 한의사를 위한 기본 한약처방 강의 2판','트리거포인트 침치료','KCD 한방내과 진찰진단 가이드라인',\
          '실전한약가이드','음양승강으로 해석하는 사상의학: 생리병리','藥徵, 약의 징표','증보운곡본초학','통증치료를 위한 근육 초음파와 주사 테크닉'],
        7:['오국통 온병명방'],
        9:['병태생리 Visual map','NEO 인턴 핸드북','보험한약 브런치 the # 2판 개정판','Kendall 자세와 통증치료에 있어서 근육의 기능과 검사 5판',\
          '사상방 사용설명서','실전한약가이드','일차진료 한의사를 위한 보험한약입문 - 둘째 판','증보운곡본초학'],
        10:['한눈에 보는 스트레칭 해부학'],
        11:['임산부에게 사용할 수 있는 한방처방'],
        12:['임산부에게 사용할 수 있는 한방처방'],
        13:['MRI 자신감 키우기_족부편'],
        14:['장골의 PI 변위는 없다'],
        15:['윤상훈·권병조의 알짜 근육학','임상 한의사를 위한 기본 한약처방 강의 2판','KCD 한방내과 진찰진단 가이드라인','트리거포인트 침치료',\
           '음양승강으로 해석하는 사상의학: 생리병리','침의 과학적 접근의 이해','실전한약가이드','임산부에게 사용할 수 있는 한방처방','한눈에 보는 스트레칭 해부학',\
           'MRI 자신감 키우기_족부편'],
        16:['환자상담의 달인','병의원 경영과 자산 관리 클리닉','우리 병원의 문제? 현장에서 답을 찾다!','근육학','스파이랄 및 키네지오 테이핑',\
           '요양병원 주치의 진료핵심'],
        17:['오당 본초강론','운동기능장애 치료 매뉴얼','K. 한의학 임상총론','한방 활용 가이드','최강통증매선','암 치료에 이용되는 천연약물',\
           '왕문원 임상 평형침법','중국 왕문원 평형침구학'],
        18:['초음파 가이드 근골격계 통증 치료의 정석'],
        19:['초음파 가이드 근골격계 통증 치료의 정석','섭혜민 명의경방험안'],
        20:['카이로프랙틱 기본테크닉론'],
        21:['흔히보는 정형외과 외래진료 가이드북'],
        22:['趙紹琴(조소금) 내과학','한의학 상담','숨찬 세상, 호흡기를 편하게',\
         '의학심오(醫學心悟)','안면마비 침구치료','중경서 독법 강해(상,하) /개정판'],
        23:['선생님, 이제 그만 저 좀 포기해 주세요','한의학 상담','숨찬 세상, 호흡기를 편하게',\
        '의학심오(醫學心悟)','중경서 독법 강해(상,하) /개정판','안면마비 침구치료'],
     24:['황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼','황황교수의 개원 한의사를 위한 상한금궤 처방 강의록',\
        '선생님, 이제 그만 저 좀 포기해 주세요'],\
     25:['황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼',\
       '황황교수의 개원 한의사를 위한 상한금궤 처방 강의록','약침의 정석 –통증편','갑상선 진료 완전정복',\
       '신경학 증상의 감별법','이것이 알고싶다! 당뇨병진료','어지럼질환의 진단과 치료','증례와 함께 하는 한약처방',\
       '뇌의학의 첫걸음','HAPPY 소아청소년 진료'],\
     26:['약침의 정석 –통증편','갑상선 진료 완전정복','신경학 증상의 감별법',\
       '증례와 함께 하는 한약처방','이것이 알고싶다! 당뇨병진료','HAPPY 소아청소년 진료','어지럼질환의 진단과 치료',\
       '뇌의학의 첫걸음','실전, 임상한의학 내과질환을 중심으로','실전, 임상한의학 알레르기질환','침구대성','평주온열경위'],
     27:['침구과 진료매뉴얼','실전, 임상한의학 내과질환을 중심으로','실전, 임상한의학 알레르기질환','내과학 5권세트','한방순환 신경내과학',\
        '침구대성'],
     28:['감별진단의 정석','기본통증진료학','약처방의 정석 (1, 2권 세트)','QBook: Case based Review',\
         'SMART 내과 1권 : 바이탈, 감염, 종양, 류마티스','일차진료아카데미 처방가이드'],
     29:['비만문답','사암침의 해석과 임상'],
     30:['플로차트 정형외과 진단','침구과 진료매뉴얼','내과학 5권세트','한방순환 신경내과학'],
     31:['외래에서 꼭 알아야 할 통증증후군 137가지'],
     32:['SMART 기본 일차진료매뉴얼 3판(세트)','SMART 소아진료매뉴얼 3판','SMART 응급진료매뉴얼(세트)'],
     33:['SMART 기본 일차진료매뉴얼 3판(세트)','SMART 소아진료매뉴얼 3판','SMART 응급진료매뉴얼(세트)'],
     34:['초음파 유도하 침 시술 가이드북'],
     35:['영어 진료 가이드북','초음파 유도하 침 시술 가이드북'],
     36:['영어 진료 가이드북','소아피부질환해설'],
     37:['소아피부질환해설','醫學心悟(의학심오) 톺아보기'],}

    promotion_item_list = []
    for promotion_items in promotion_dict.values():
        for item in promotion_items:
            promotion_item_list.append(item)

    # set(promotion_item_list), len(set(promotion_item_list))
    
    preprocess_promotion_df = promotion_book_df[~((promotion_book_df['name_x'].str.contains('침의 과학적 접근과 임상활용')) & \
                            (promotion_book_df['date_paid_week']==3))]
    preprocess_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('의학심오')) & \
                                (preprocess_promotion_df['date_paid_week']==22))]
    preprocess_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('의학심오')) & \
                                (preprocess_promotion_df['date_paid_week']==23))]
    preprocess_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('약처방의 정석')) & \
                                (preprocess_promotion_df['date_paid_week']==28))]
    preprocess_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('초음파 유도하 침')) & \
                                (preprocess_promotion_df['date_paid_week']==34))]
    preprocess_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('초음파 유도하 침')) & \
                                (preprocess_promotion_df['date_paid_week']==34))]
    preprocess_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('영어 진료 가이드북')) & \
                                (preprocess_promotion_df['date_paid_week']==35))]
    preprocess_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('영어 진료 가이드북')) & \
                                (preprocess_promotion_df['date_paid_week']==36))]
    all_promotion_df = preprocess_promotion_df[~((preprocess_promotion_df['name_x'].str.contains('의학심오')) & \
                                (preprocess_promotion_df['date_paid_week']==37))]

    for key,value in promotion_dict.items():
        all_promotion_df = all_promotion_df[~((all_promotion_df['name_x'].isin(value)) & (all_promotion_df['date_paid_week']==key))]
    
    return all_promotion_df

In [7]:
df['date_paid'].max()

Timestamp('2022-09-13 08:59:21.151000+0000', tz='UTC')

In [8]:
split_day_list =  []
test_day_list = []

# 1일 test 날짜 확인
last_date_state = '2022-09-13'
split_day = pd.to_datetime(last_date_state)-relativedelta(months=3)
test_day = pd.to_datetime(last_date_state)
'''
마지막 날짜에서 개월 수를 자름 -> split_day
months 만 바꾸면 21번 분량이 나옴
'''

for i in range(0,21):
    sp_day = str((split_day-timedelta(days=i)).to_pydatetime().date())
    tt_day  = str((test_day-timedelta(days=i)).to_pydatetime().date())
    split_day_list.append(sp_day)
    test_day_list.append(tt_day)

In [9]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]
    
    def _entropy_diversity(self,rec_list):
        import six
        import math
        
        topn = len(rec_list[0]['items'])
        users = [i.get('id',None) for i in rec_list]
        sz = float(len(users)) * topn
        freq = {}
        for rec in rec_list:
            for r in rec['items']:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)
        ent = self._entropy_diversity(rec_list)
        
        return ndcg_score, ent

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score, ent_score = self._eval(gt_list, rec_list)
            print(f"NDCG: {ndcg_score:.6}")
            print(f"Entropy Diversity: {ent_score:.6} ")
        except Exception as e:
            print(e)


In [10]:
def module(df:pd.DataFrame(), split_date, test_date, all_df)->pd.DataFrame():
    
    # paid orders만 가져오기
    df['date_paid'] = pd.to_datetime(df['date_paid'])
    df_only_paid = df[~df['date_paid'].isna()]
    # 3개월치 데이터만 가져오기
    df_date = df_only_paid[df_only_paid['date_paid'] >= split_date]
    # 취소 안된 것만 가져오기
    complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
    # 도서 카테고리만 가져오기
    only_book = complete_df[complete_df['name'] == '도서']

    # 유저가 중복으로 아이템 구매 삭제
    df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
    df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)
    
    # medirecommend 만들기
    df = df.dropna(subset=['product_ids','name_x'])

    # 나오는 개월 수 적기
    date_state = "2022-06-13"
    # paid orders만 가져오기
    df['date_paid'] = pd.to_datetime(df['date_paid'])
    df_only_paid = df[~df['date_paid'].isna()]
    # 취소 안된 것만 가져오기
    complete_df = df_only_paid[(df_only_paid['paid'] == True) & (df_only_paid['cancelled']==False)]
    # 도서 카테고리만 가져오기
    only_book = complete_df[complete_df['name'] == '도서']

    # 유저가 중복으로 아이템 구매 삭제
    df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
    df_sort = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)
    df_sort = product_name_fill(df_sort)
    df_sort = df_sort.drop_duplicates(subset=['customer_id','product_ids']).reset_index(drop=True)
    
    # 변수 처리한 기간 데이터만 가져오기
    df_book = df_sort[df_sort['date_paid'] >= split_date].reset_index(drop=True)

    # 마지막 3주 제외한 medirecommend 만들기
    mediprediction_all_df = df_sort[df_sort['date_paid'] < '2022-08-23'].reset_index(drop=True)
    
    train_before = df_book[df_book['date_paid'] < test_date]
    train = promotion_proprof(train_before)
    test = df_book[df_book['date_paid'].dt.date == pd.to_datetime(test_date)]
    
    
    # test 만 있는 item 제거
    only_test_items = set(test.product_ids.unique())-set(train.product_ids.unique())
    if_prepro_test = test[~test['product_ids'].isin(only_test_items)]
    
    
    PdIds = train.product_ids.unique()

    PdIdToIndex = {}
    indexToPdId = {}

    colIdx = 0

    for PdId in PdIds:
        PdIdToIndex[PdId] = colIdx
        indexToPdId[colIdx] = PdId
        colIdx += 1

    userIds = train.customer_id.unique()

    userIdToIndex = {}
    indexToUserId = {}

    rowIdx = 0

    for userId in userIds:
        userIdToIndex[userId] = rowIdx
        indexToUserId[rowIdx] = userId
        rowIdx += 1
    rows = []
    cols = []
    vals = []

    for row in train.itertuples():
        rows.append(userIdToIndex[row.customer_id])
        cols.append(PdIdToIndex[row.product_ids])
        vals.append(1)

    purchase_sparse = sp.csr_matrix((vals, (rows, cols)), shape=(rowIdx,colIdx))

    matrix = purchase_sparse.todense()

    medistream_prediction_df = mediprediction_all_df[['date_created','regular_price','sale_price','three_months','product_ids','name_x']]
    medistream_prediction_preprop_df = medistream_prediction_df.drop_duplicates(subset=['product_ids'], ignore_index=True)
    medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])    
    
    als_model = ALS(factors=20,regularization=0.01, iterations = 50, random_state=42)
    alsmodel = als_model.fit(purchase_sparse)
    lmf_model = LMF(factors=20, regularization=0.001, iterations = 20, random_state=42)
    lmfmodel = lmf_model.fit(purchase_sparse)
    
    most_popular = mediprediction_all_df.groupby(['product_ids']).count()['customer_id'].reset_index()
    most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index
    

        # test 예측값, 이미 구매 했을 경우 제외
    als_predict_list = []
    for user_id in test['customer_id'].unique():
        try:
            result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
            als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
        except:
            train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
            als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(als_predict_list):
        als_predict_list[idx]['items'] = pred_list['items'][:100]

    

        # test 예측값
    lmf_predict_list = []
    for user_id in test['customer_id'].unique():
        try:
            result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
            lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
        except:
            train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
            lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(lmf_predict_list):
        lmf_predict_list[idx]['items'] = pred_list['items'][:100]

    

        # test 예측값, 이미 구매 했을 경우 제외
    predict_popular_list = []
    for user_id in test['customer_id'].unique():
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        predict_popular_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(predict_popular_list):
        predict_popular_list[idx]['items'] = pred_list['items'][:100]
        
    # real test 
    ground_trues = []
    for user_id in test['customer_id'].unique():
        ground_trues.append({'id': user_id,\
        'items':list(test[test['customer_id']==user_id].product_ids)
        })
    # ALS 
    evaluator = CustomEvaluator()
    als = evaluator._eval(ground_trues, als_predict_list)
    
    # LMF 
    evaluator = CustomEvaluator()
    lmf = evaluator._eval(ground_trues, lmf_predict_list)
    # MP
    evaluator = CustomEvaluator()
    mp = evaluator._eval(ground_trues, predict_popular_list)
    
    # 인기도순
    medistream_popular_list = medistream_prediction_preprop_df.sort_values(by='three_months', ascending=False).index
    # 최신순
    medistream_latest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=False).index
    # 오랜된 순
    medistream_oldest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=True).index
    # 높은 가격 순
    medistream_high_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=False).index
    # 낮은 가격 순
    medistream_low_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=True).index
    # 이름 순
    medistream_name_sort_list = medistream_prediction_preprop_df.sort_values(by='name_x',ascending=True).index

    def medistream_prediction_method(predict_num:int ,medi_predict_list:list)->list:
        medistream_predict_list = []
        for user_id in test['customer_id'].unique():
            medistream_predict_list.append({'id':user_id ,'items':[medistream_prediction_preprop_df.product_ids.loc[num] \
                                                                           for num in medi_predict_list]})

        # 100 개만 예측하기
        for idx, pred_list in enumerate(medistream_predict_list):
            medistream_predict_list[idx]['items'] = pred_list['items'][:predict_num]

        return medistream_predict_list
    
    medistream_predict_popular_list = medistream_prediction_method(100, medistream_popular_list)
    medistream_predict_latest_list = medistream_prediction_method(100, medistream_latest_list)
    medistream_predict_oldest_list = medistream_prediction_method(100, medistream_oldest_list)
    medistream_predict_high_price_list = medistream_prediction_method(100, medistream_high_price_list)
    medistream_predict_low_price_list = medistream_prediction_method(100, medistream_low_price_list)
    medistream_predict_name_sort_list = medistream_prediction_method(100, medistream_name_sort_list)
    
    def medistream_prediction(ground_trues:list, predict_list:list):
        evaluator = CustomEvaluator()
        ndcg, entropy = evaluator._eval(ground_trues, predict_list)

        assert len(predict_list) == len(ground_trues)

        cnt = 0
        for gt, pred_list in zip(ground_trues, predict_list):
            for pred in pred_list['items']:
                if pred in gt['items']:
                    cnt += 1
        return ndcg, entropy, cnt
    
    medistream_predict_score = {'medistream_predict':['medi_popular','latest','oldest','high_price','low_price','name_sort'], \
                                'ndcg':[], 'entropy':[], 'cnt':[]}

    medistream_predict_list = [medistream_predict_popular_list, medistream_predict_latest_list, medistream_predict_oldest_list,\
                              medistream_predict_high_price_list, medistream_predict_low_price_list, medistream_predict_name_sort_list]

    for medistream_predict in medistream_predict_list:
        ndcg, entropy, cnt = medistream_prediction(ground_trues, medistream_predict)
        medistream_predict_score['ndcg'].append(ndcg)
        medistream_predict_score['entropy'].append(entropy)
        medistream_predict_score['cnt'].append(cnt)

        
######## hyper parameter
    als_mf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

    factors = [5,10,15,20]
    regularizations = [0.01,0.005]
    iterations = [5,10,15,20,25,30,40,50]

    for factor in factors:
        for regularization in regularizations:
            for iteration in iterations:
                als_model = ALS(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
                als_model.fit(purchase_sparse, show_progress=False)

                # 신규 유저인 경우 mp로 넣기
                # 전체 도서에 대한 판매 만큼 정렬 후 넣기
                most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

                # test 예측값, 이미 구매 했을 경우 제외
                als_predict_list = []
                for user_id in test['customer_id'].unique():
                    try:
                        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                        als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                    except:
                        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                        als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                            ]})

                # 100 개만 예측하기
                for idx, pred_list in enumerate(als_predict_list):
                    als_predict_list[idx]['items'] = pred_list['items'][:100]

                # ALS 
                evaluator = CustomEvaluator()
                ndcg, entropy = evaluator._eval(ground_trues, als_predict_list)

                als_mf_hyper_parameter['factor'].append(factor)
                als_mf_hyper_parameter['regularization'].append(regularization)
                als_mf_hyper_parameter['iteration'].append(iteration)
                als_mf_hyper_parameter['NDCG'].append(ndcg)
                als_mf_hyper_parameter['entropy'].append(entropy)
                
                
    lmf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

    factors = [5,10,15,20]
    regularizations = [0.01,0.005]
    iterations = [5,10,15,20,25,30,40,50]

    for factor in factors:
        for regularization in regularizations:
            for iteration in iterations:
                lmf_model = LMF(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
                lmf_model.fit(purchase_sparse, show_progress=False)

                # 신규 유저 mp로 넣기
                most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

                # test 예측값
                lmf_predict_list = []
                for user_id in test['customer_id'].unique():
                    try:
                        result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                        lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                    except:
                        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                        lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                            ]})

                # 100 개만 예측하기
                for idx, pred_list in enumerate(lmf_predict_list):
                    lmf_predict_list[idx]['items'] = pred_list['items'][:100]

                # LMF
                evaluator = CustomEvaluator()
                ndcg, entropy = evaluator._eval(ground_trues, lmf_predict_list)

                lmf_hyper_parameter['factor'].append(factor)
                lmf_hyper_parameter['regularization'].append(regularization)
                lmf_hyper_parameter['iteration'].append(iteration)
                lmf_hyper_parameter['NDCG'].append(ndcg)
                lmf_hyper_parameter['entropy'].append(entropy)
                
#     return als_mf_hyper_parameter, lmf_hyper_parameter, medistream_predict_score

    all_prediction_df = {'first_day':[],'last_day':[],'train_데이터수':[],'train_유저수':[],'test_데이터수':[],\
        'test_유저수':[],'test_신규유저수':[],'test_신규아이템수':[],'원본_test수':[],'전처리진행test수':[],\
        'als_mf':[],'lmf':[],'mp':[],'medi_popular':[],'latest':[],\
        'oldest':[],'high_price':[],'low_price':[],'name_sort':[],\
         'als_mf_entropy':[],'lmf_entropy':[],'mp_entropy':[],'medi_popular_entropy':[],'latest_entropy':[],\
         'oldest_entropy':[],'high_price_entropy':[],'low_price_entropy':[],'name_sort_entropy':[]}
    medistream_predict_df = pd.DataFrame(medistream_predict_score)

    all_prediction_df['first_day'].append(str(datetime.date(train['date_paid'].min()))+' '+str(datetime.date(train['date_paid'].max())))
    all_prediction_df['last_day'].append(str(datetime.date(test['date_paid'].min()))+' '+str(datetime.date(test['date_paid'].max())))
    all_prediction_df['train_데이터수'].append(len(train))
    all_prediction_df['train_유저수'].append(len(set(train.customer_id)))
    all_prediction_df['test_데이터수'].append(len(test))
    all_prediction_df['test_유저수'].append(len(set(test.customer_id)))
    all_prediction_df['test_신규유저수'].append(len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))
    all_prediction_df['test_신규아이템수'].append(len(set(test.product_ids.unique())-set(train.product_ids.unique())))
    all_prediction_df['원본_test수'].append(len(test))
    all_prediction_df['전처리진행test수'].append(len(if_prepro_test))

    # ndcg
    all_prediction_df['als_mf'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
    all_prediction_df['lmf'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
    all_prediction_df['mp'].append(evaluator._eval(ground_trues, predict_popular_list)[0])
    all_prediction_df['medi_popular'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['ndcg'])
    all_prediction_df['latest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['ndcg'])
    all_prediction_df['oldest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['ndcg'])
    all_prediction_df['high_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['ndcg'])
    all_prediction_df['low_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['ndcg'])
    all_prediction_df['name_sort'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['ndcg'])

#     entropy
    all_prediction_df['als_mf_entropy'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
    all_prediction_df['lmf_entropy'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
    all_prediction_df['mp_entropy'].append(evaluator._eval(ground_trues, predict_popular_list)[1])
    all_prediction_df['medi_popular_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['entropy'])
    all_prediction_df['latest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['entropy'])
    all_prediction_df['oldest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['entropy'])
    all_prediction_df['high_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['entropy'])
    all_prediction_df['low_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['entropy'])
    all_prediction_df['name_sort_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['entropy'])

    print('train 총 기간:',train['date_paid'].max()-train['date_paid'].min())
    print('test 총 기간:',test['date_paid'].max()-test['date_paid'].min())
    
    return pd.DataFrame(all_prediction_df)

In [11]:
from tqdm import tqdm

twenty_df_list = []
for sp_day, tt_day in tqdm(zip(split_day_list,test_day_list)):
    module_df = module(df, sp_day,tt_day, all_df)
    twenty_df_list.append(module_df)
twenty_df = pd.concat(twenty_df_list, ignore_index=True)
twenty_df

0it [00:00, ?it/s]<ipython-input-10-489f192e97f4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_paid'] = pd.to_datetime(df['date_paid'])
<ipython-input-10-489f192e97f4>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

1it [06:35, 395.99s/it]

train 총 기간: 91 days 23:09:58.835000
test 총 기간: 0 days 08:44:11.628000


...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

21it [2:26:15, 417.88s/it]

train 총 기간: 91 days 13:00:55.657000
test 총 기간: 0 days 13:49:59


,first_day,last_day,train_데이터수,train_유저수,test_데이터수,test_유저수,test_신규유저수,test_신규아이템수,원본_test수,전처리진행test수,als_mf,lmf,mp,medi_popular,latest,oldest,high_price,low_price,name_sort,als_mf_entropy,lmf_entropy,mp_entropy,medi_popular_entropy,latest_entropy,oldest_entropy,high_price_entropy,low_price_entropy,name_sort_entropy
0,2022-06-13 2022-09-12,2022-09-13 2022-09-13,1994,1069,16,13,9,1,16,15,0.357634,0.421923,0.338264,0.237268,0.063234,0.100933,0.037457,0.052212,0.032969,5.126065,5.058989,4.616045,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
1,2022-06-12 2022-09-11,2022-09-12 2022-09-12,1995,1068,15,11,6,0,15,15,0.270971,0.316789,0.235122,0.303531,0.070774,0.133306,0.085115,0.018371,0.043540,5.233183,5.191451,4.630434,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
2,2022-06-11 2022-09-10,2022-09-11 2022-09-11,2005,1070,4,2,1,1,4,3,0.459785,0.455621,0.403287,0.459860,0.142857,0.000000,0.193426,0.064475,0.057606,4.979470,4.986401,4.605170,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
3,2022-06-10 2022-09-09,2022-09-10 2022-09-10,2019,1081,2,1,0,0,2,2,0.528722,0.444068,0.213849,0.233596,0.000000,0.200249,0.097267,0.103385,0.092089,4.605170,4.605170,4.605170,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
4,2022-06-09 2022-09-08,2022-09-09 2022-09-09,2048,1100,6,4,1,0,6,6,0.260335,0.259871,0.192983,0.163724,0.025846,0.210808,0.089909,0.000000,0.000000,5.251684,5.245143,4.647541,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
5,2022-06-08 2022-09-07,2022-09-08 2022-09-08,2049,1102,99,81,65,1,99,35,0.102177,0.115494,0.103061,0.077664,0.021982,0.036035,0.015614,0.021179,0.021081,5.024994,4.965200,4.619066,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
6,2022-06-07 2022-09-06,2022-09-07 2022-09-07,2046,1106,33,22,16,2,33,27,0.177411,0.227292,0.181032,0.228050,0.058293,0.102501,0.014734,0.065778,0.034292,5.109293,5.046275,4.617448,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
7,2022-06-06 2022-09-05,2022-09-06 2022-09-06,2045,1106,58,50,35,1,58,52,0.054782,0.076745,0.047769,0.254240,0.027591,0.126021,0.001807,0.011314,0.001842,5.146811,5.085240,4.618063,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
8,2022-06-05 2022-09-04,2022-09-05 2022-09-05,2036,1100,24,18,14,1,24,20,0.289145,0.317646,0.271472,0.249567,0.102507,0.069962,0.030700,0.051831,0.048461,5.038665,4.989065,4.619569,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
9,2022-06-04 2022-09-03,2022-09-04 2022-09-04,2024,1096,20,15,6,1,20,18,0.280681,0.333022,0.297913,0.226832,0.074468,0.163072,0.113595,0.011046,0.076531,5.343105,5.295753,4.626884,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
